In [16]:
import os
import torch
import torchvision
import numpy as np
from torchvision import models, transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Define dataset paths
data_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')



In [17]:
# Define data transformations
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the datasets
train_data = ImageFolder(train_dir, transform=data_transforms)
test_data = ImageFolder(test_dir, transform=data_transforms)

# Split the train_data into train and validation sets
train_size = int(0.8 * len(train_data))
val_size = len(train_data) - train_size
train_data, val_data = random_split(train_data, [train_size, val_size])

# Create DataLoaders for the datasets
batch_size = 16
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Initialize the ResNet-152 model
model = models.resnet152(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop with progress bar, loss, accuracy, validation loss, and validation accuracy
epochs = 15
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")

    for inputs, labels in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

        # Update the progress bar
        progress_bar.set_postfix(loss=loss.item(), acc=torch.true_divide(running_corrects, ((epoch * len(train_loader.dataset)) + inputs.size(0))).item())

    # Calculate epoch loss and accuracy
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = running_corrects.double() / len(train_loader.dataset)

    # Validation loop
    model.eval()
    val_running_loss = 0.0
    val_running_corrects = 0

    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        with torch.no_grad():
            outputs = model(inputs)
            val_loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)

        val_running_loss += val_loss.item() * inputs.size(0)
        val_running_corrects += torch.sum(preds == labels.data)

    # Calculate validation loss and accuracy
    val_epoch_loss = val_running_loss / len(val_loader.dataset)
    val_epoch_acc = val_running_corrects.double() / len(val_loader.dataset)

    # Print epoch results
    print(f"Epoch {epoch+1}/{epochs}: Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")





Epoch 1/15: 100%|██████████| 261/261 [01:58<00:00,  2.19it/s, acc=327, loss=0.031]   


Epoch 1/15: Train Loss: 0.1415, Train Acc: 0.9415, Val Loss: 0.0536, Val Acc: 0.9799


Epoch 2/15: 100%|██████████| 261/261 [01:59<00:00,  2.19it/s, acc=0.984, loss=0.301]   


Epoch 2/15: Train Loss: 0.0406, Train Acc: 0.9868, Val Loss: 0.0333, Val Acc: 0.9856


Epoch 3/15: 100%|██████████| 261/261 [02:01<00:00,  2.15it/s, acc=0.497, loss=0.000853]


...

Epoch 13/15: Train Loss: 0.0030, Train Acc: 0.9993, Val Loss: 0.0394, Val Acc: 0.9866


Epoch 14/15: 100%|██████████| 261/261 [02:00<00:00,  2.16it/s, acc=0.0769, loss=0.00084] 


Epoch 14/15: Train Loss: 0.0013, Train Acc: 0.9998, Val Loss: 0.0287, Val Acc: 0.9904


Epoch 15/15: 100%|██████████| 261/261 [01:59<00:00,  2.19it/s, acc=0.0714, loss=1.86e-5] 


Epoch 15/15: Train Loss: 0.0004, Train Acc: 1.0000, Val Loss: 0.0316, Val Acc: 0.9875


In [18]:
# Test the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on test set: {accuracy:.2f}%")

# Save the trained model
torch.save(model.state_dict(), '/kaggle/working/resnet152_chest_xray_pneumonia.pth')

Accuracy on test set: 87.18%


RESNET 200 Custom Model

In [24]:
import os
import torch
import torchvision
import numpy as np
from torchvision import models, transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torchvision.models import resnet

# Custom ResNet-200 model
class CustomBottleneck(resnet.Bottleneck):
    expansion = 4

def _resnet(arch, block, layers, pretrained, progress, **kwargs):
    model = resnet.ResNet(block, layers, **kwargs)
    return model

def resnet200(pretrained=False, progress=True, **kwargs):
    return _resnet('resnet200', CustomBottleneck, [3, 24, 36, 3], pretrained, progress, **kwargs)

# Define dataset paths
data_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')

# Define data transformations
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the datasets
train_data = ImageFolder(train_dir, transform=data_transforms)
test_data = ImageFolder(test_dir, transform=data_transforms)

# Split the train_data into train and validation sets
train_size = int(0.8 * len(train_data))
val_size = len(train_data) - train_size
train_data, val_data = random_split(train_data, [train_size, val_size])

# Create DataLoaders for the datasets
batch_size = 16
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Initialize the custom ResNet-200 model
model = resnet200()
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training and validation loop
epochs = 15
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")

    for inputs, labels in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

        # Update the progress bar
        progress_bar.set_postfix(loss=loss.item(), acc=torch.true_divide(running_corrects, ((epoch * len(train_loader.dataset)) + inputs.size(0))).item())

    # Calculate epoch loss and accuracy
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = running_corrects.double() /len(train_loader.dataset)
# Validation loop
model.eval()
val_running_loss = 0.0
val_running_corrects = 0

for inputs, labels in val_loader:
    inputs, labels = inputs.to(device), labels.to(device)

    with torch.no_grad():
        outputs = model(inputs)
        val_loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)

    val_running_loss += val_loss.item() * inputs.size(0)
    val_running_corrects += torch.sum(preds == labels.data)

# Calculate validation loss and accuracy
val_epoch_loss = val_running_loss / len(val_loader.dataset)
val_epoch_acc = val_running_corrects.double() / len(val_loader.dataset)

# Print epoch results
print(f"Epoch {epoch+1}/{epochs}: Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

# Test the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on test set: {accuracy:.2f}%")

# Save the trained model
torch.save(model.state_dict(), '/kaggle/working/resnet200_chest_xray_pneumonia.pth')






Epoch 15/15: 100%|██████████| 261/261 [02:19<00:00,  1.87it/s, acc=0.0695, loss=0.298]   


Epoch 15/15: Train Loss: 0.0783, Train Acc: 0.9734, Val Loss: 0.0894, Val Acc: 0.9751
Accuracy on test set: 83.01%


Report for Resnet 200

In [34]:
# Custom ResNet-200 model definition
class CustomBottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1, base_width=64, dilation=1, norm_layer=None):
        super(CustomBottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = norm_layer(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=dilation, bias=False, groups=groups, dilation=dilation)
        self.bn2 = norm_layer(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = norm_layer(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

def _resnet(arch, block, layers, pretrained, progress, **kwargs):
    model = torchvision.models.ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch], progress=progress)
        model.load_state_dict(state_dict)
    return model

def resnet200(pretrained=False, progress=True, **kwargs):
    return _resnet('resnet200', CustomBottleneck, [3, 24, 36, 3], pretrained, progress, **kwargs)

# Load the custom ResNet-200 model
model = resnet200(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model.load_state_dict(torch.load('resnet200_chest_xray_pneumonia.pth'))
model = model.to(device)
# Test the model
model.eval()
all_labels = []
all_preds = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

# Calculate accuracy
accuracy = np.mean(np.array(all_labels) == np.array(all_preds)) * 100
print(f"Accuracy on test set: {accuracy:.2f}%")

# Generate classification report and confusion matrix
report = classification_report(all_labels, all_preds, target_names=['0', '1'])
conf_matrix = confusion_matrix(all_labels, all_preds)

print("Classification Report for RESNET 200:")
print(report)

print("Confusion Matrix:")
print(conf_matrix)


Accuracy on test set: 83.01%
Classification Report for RESNET 200:
              precision    recall  f1-score   support

           0       0.98      0.56      0.71       234
           1       0.79      0.99      0.88       390

    accuracy                           0.83       624
   macro avg       0.88      0.78      0.80       624
weighted avg       0.86      0.83      0.82       624

Confusion Matrix:
[[131 103]
 [  3 387]]


Report for RESNET 152

In [29]:
import torch
import os
import numpy as np
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from sklearn.metrics import classification_report, confusion_matrix

# Define dataset paths
data_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray'
test_dir = os.path.join(data_dir, 'test')

# Define data transformations
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the test dataset
test_data = ImageFolder(test_dir, transform=data_transforms)
test_loader = DataLoader(test_data, batch_size=16, shuffle=False)

# Load the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = models.resnet152(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model.load_state_dict(torch.load('resnet152_chest_xray_pneumonia.pth'))
model = model.to(device)

# Test the model
model.eval()
all_labels = []
all_preds = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

# Calculate accuracy
accuracy = np.mean(np.array(all_labels) == np.array(all_preds)) * 100
print(f"Accuracy on test set: {accuracy:.2f}%")

# Generate classification report and confusion matrix
report = classification_report(all_labels, all_preds, target_names=['0', '1'])
conf_matrix = confusion_matrix(all_labels, all_preds)

print("Classification Report for RESNET 152:")
print(report)

print("Confusion Matrix:")
print(conf_matrix)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Accuracy on test set: 87.18%
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.66      0.79       234
           1       0.83      1.00      0.91       390

    accuracy                           0.87       624
   macro avg       0.91      0.83      0.85       624
weighted avg       0.89      0.87      0.86       624

Confusion Matrix:
[[155  79]
 [  1 389]]
